<hr>
<font size="6"> <b> Updating Image Services </b></font>

<font size="5"> Update Image Service with Archived Maps in S3 Bucket </font>

<br>
<font size="4"> <b> HydroSAR Transition Workshop:</b> HydroSAR Data Products and Web Application Training <br>
<font size="4"> <b> PI:</b> Franz J. Meyer <br>
<br>
<br>

Import relevant python packages.

In [ ]:
import datetime
import os
import subprocess

import arcpy

Set up important paths, a date-time variable, and tags we will use.

In [ ]:
batch_dir = r'C:\Users\asf\asf_services\batchFiles'
scratch_gdb = r"C:\Users\asf\asf_services\ImageServerScratch.gdb"

today = datetime.datetime.now(datetime.timezone.utc).strftime("%y%m%d_%H%M")

s3tag = 'HKHwatermaps'
projtag = 'HKH'

crftag_wm = projtag+'_WatermapExtent'
crftag_rgb = projtag+'_RGB'
crftag_rtc = projtag+'_RTC'

Set general variables used every time the script runs. Then, set variables that will used for today's dataset.
Name the crfs for each dataset for today's date.

In [ ]:
genvars = [r'set ppath="C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\python.exe"',
           r'set mdcspath=C:\Users\asf\asf_services',
           r'set cachepath=C:\Users\asf\asf_services\PixelCache']

vars = [r'set gdbwks=C:\Users\asf\asf_services\MD''\\'
        +projtag+'\\'+projtag+'_'+today+'.gdb',
        r'set acspath=C:\Users\asf\NASA_Disasters_AWS.acs''\\',
        r'set s3tag='+s3tag]

crf_wm = r'set outcrf=C:\Users\asf\NASA_Disasters_AWS.acs\esri''\\'\
         +crftag_wm+'_'+today+'.crf'
crf_rgb = r'set outcrf=C:\Users\asf\NASA_Disasters_AWS.acs\esri''\\'\
          +crftag_rgb+'_'+today+'.crf'
crf_rtc = r'set outcrf=C:\Users\asf\NASA_Disasters_AWS.acs\esri''\\'\
          +crftag_rtc+'_'+today+'.crf'

Set up variables for watermap, RGB, and RTC datasets.

In [ ]:
vars_wm = [r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\wm_mosaic.xml '
           r'-m:%gdbwks%\watermap_extent -s:%acspath%%s3tag%\ -p:%cachepath%\$cachelocation '
           r'-p:USE_PIXEL_CACHE$pixelcache -p:%s3tag%$stag -c:CM+AF+AR+UpdateFieldsWM+BF+BB+SP+CC',
           r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\wm_overviews.xml '
           r'-m:%gdbwks%\watermap_extent -s:%outcrf% -c:SE+CRA+AR+UpdateOverviewFields -p:%outcrf%$outcrf']

vars_rgb = [r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\rgb_mosaic.xml '
            r'-m:%gdbwks%\rgb -s:%acspath%%s3tag%\ -p:%cachepath%\$cachelocation -p:USE_PIXEL_CACHE$pixelcache '
            r'-p:%s3tag%$stag -c:CM+AF+AR+UpdateFieldsRGB+BF+BB+SP+CC',
            r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\rgb_overviews.xml '
            r'-m:%gdbwks%\rgb -s:%outcrf% -c:SE+CRA+AR+UpdateOverviewFields -p:%outcrf%$outcrf']

vars_rtc = [r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\rtc_mosaic_s1.xml '
            r'-m:%gdbwks%\sar_s1 -s:%acspath%%s3tag%\ -p:%cachepath%\$cachelocation -p:USE_PIXEL_CACHE$pixelcache '
            r'-c:CM+AF+AR+UpdateFieldsRTC+BF+BB+SP+CC',
            r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\rtc_mosaic_comp.xml '
            r'-m:%gdbwks%\sar_comp -s:%gdbwks%\sar_s1 -p:CompositeVV_VH_32.art.xml$art '
            r'-p:%s3tag%$stag -c:CM+AR+UpdateNameFieldRTC+BB+SP+CC',
            r'%ppath% %mdcspath%\scripts\MDCS.py -i:%mdcspath%\Parameter\Config\rtc_overviews.xml '
            r'-m:%gdbwks%\sar_comp -s:%outcrf% -c:SE+CRA+AR+UpdateOverviewFields -p:%outcrf%$outcrf']

Loop through batch files and write general reference variables, output geodatabases, AWS bucket paths, and service-specific variables and batch commands.

In [ ]:
batfile_wm = batch_dir + '\HKH_wm.bat'
batfile_rgb = batch_dir + '\HKH_rgb.bat'
batfile_rtc = batch_dir + '\HKH_rtc.bat'

batfiles = [batfile_wm, batfile_rgb, batfile_rtc]

for bat in batfiles:
    with open(bat, 'w') as f:
        # write general reference variables
        for gv in genvars:
            print(gv, file=f)
        print('\nREM ---------- Set output gdb and source s3 bucket-----', file=f)
        # write gdb output and aws connection specifications
        for v in vars:
            print(v, file=f)
        # write the service-specific variables and batch commands
        if bat == 'HKH_wm.bat':
            print(crf_wm, file=f)
            print('\nREM ---------- Using ACS File -------------------------', file=f)
            for vb in vars_wm:
                print(vb, file=f)
        elif bat == 'HKH_rgb.bat':
            print(crf_rgb, file=f)
            print('\nREM ---------- Using ACS File -------------------------', file=f)
            for vb in vars_rgb:
                print(vb, file=f)
        elif bat == 'HKH_rtc.bat':
            print(crf_rtc, file=f)
            print('\nREM ---------- Using ACS File -------------------------', file=f)
            for vb in vars_rtc:
                print(vb, file=f)
        else:
            print('No valid file.')

Run watermap, RGB, and RTC batch files.

In [ ]:
subprocess.call([batfile_wm])
print('Watermap Extent mosaic dataset complete.')
subprocess.call([batfile_rgb])
print('RGB mosaic dataset complete.')
subprocess.call([batfile_rtc])
print('RTC mosaic dataset complete.')

Initialize water extent AID package, which will also set up paths we can use for the RGB and RTC AID packages.

In [ ]:
aid_path = r'C:\Users\asf\asf_services\AID''\\'+projtag
arcpy.ImportToolbox(r"C:\Users\asf\asf_services\AID\AID_GPtools\AID_Management.pyt")
gdb = r'C:\Users\asf\asf_services\MD''\\'\
        +projtag+'\\'+projtag+'_'+today+'.gdb'

print('Generating watermap extent AID package...')
md_wm = gdb+'\\'+'watermap_extent'
aid_wm = aid_path+'\\'+projtag+'_WatermapExtent_'+today+'.zmd'

Clip mosaic dataset to boundary using reference shapefile.

In [ ]:
extent_mask = r'C:\Users\hjkristenson\PycharmProjects\hyp3-nasa-disasters\image_server\asf_services\Parameter' \
              r'\Boundary\ExtentMasks.gdb\HKH_ServiceExtentMask '
arcpy.management.ImportMosaicDatasetGeometry(md_wm, "BOUNDARY", "OBJECTID", extent_mask, "FID")
print('Clipped mosaic dataset to reference shapefile.')

In [ ]:
with arcpy.EnvManager(scratchWorkspace=scratch_gdb, workspace=scratch_gdb):
    try:
        arcpy.AID.AIDISDP(md_wm, aid_wm, None)
    except:
        print("AID errors generated and ignored.")
        pass
print('Watermap extent AID package complete.')

Generate an AID package for RGB dataset.

In [ ]:
print('Generating RGB AID package...')
md_rgb = gdb+'\\'+'rgb'
aid_rgb = aid_path+'\\'+projtag+'_RGB_'+today+'.zmd'
with arcpy.EnvManager(scratchWorkspace=scratch_gdb, workspace=scratch_gdb):
    try:
        arcpy.AID.AIDISDP(md_rgb, aid_rgb, None)
    except:
        print("AID errors generated and ignored.")
        pass
print('RGB AID package complete.')

Generate an AID package for RTC dataset.

In [ ]:
print('Generating RTC AID package...')
md_rtc = gdb+'\\'+'sar_comp'
aid_rtc = aid_path+'\\'+projtag+'_RTC_'+today+'.zmd'
with arcpy.EnvManager(scratchWorkspace=scratch_gdb, workspace=scratch_gdb):
    try:
        arcpy.AID.AIDISDP(md_rtc, aid_rtc, None)
    except:
        print("AID errors generated and ignored.")
        pass
print('RTC AID package complete.')

Sign into Image Services using Enterprise credentials.

In [ ]:
import keyring
pw = keyring.get_password("portal_creds", "ASF")
arcpy.SignInToPortal(r'https://asf-daac.maps.arcgis.com/', 'ASF', pw)

Update Image Services for watermap extent, RGB, and RTC datasets.

In [ ]:
print('Updating Watermap Extent Image Service...')
arcpy.AID.MAIDIS("asf-daac", "Update Service", "HKH", "None", "HKH_Watermap_Extent", None, aid_wm, "Dedicated Instance",
                 "Watermap Extent products generated from Sentinel-1 SAR "
                 "imagery over flood-prone regions in the Hindu Kush Himalayan (HKH) region for the 2022 monsoon "
                 "season, generated by ASF.", "Imagery products processed by ASF DAAC HyP3 2022 using GAMMA software. "
                 "Contains modified Copernicus Sentinel data 2022, processed by ESA.", '',
                 False, False, True, None, None, None, None)
print('Watermap Extent Image Service updated.')
print('Updating RGB Image Service...')
arcpy.AID.MAIDIS("asf-daac", "Update Service", "HKH", "None", "HKH_RGB", None, aid_rgb, "Dedicated Instance",
                 "Sentinel-1 RGB Decomposition of RTC VV and VH "
                 "imagery over the Hindu Kush Himalayan (HKH) region for the 2022 monsoon season, starting "
                 "May 16, 2022. Blue areas have low returns in VV and VH (smooth surfaces such as calm water, "
                 "but also frozen/crusted soil or dry sand), Green areas have high returns in VH (volume scatterers "
                 "such as vegetation or some types of snow/ice), and Red areas have relatively high VV returns "
                 "and relatively low VH returns (such as urban or sparsely vegetated areas).",
                 "Imagery products processed by ASF DAAC HyP3 2022 using GAMMA software. Contains modified "
                 "Copernicus Sentinel data 2022, processed by ESA.", '',
                 False, False, True, None, None, None, None)
print('RGB Image Service updated.')
print('Updating RTC Image Service...')
arcpy.AID.MAIDIS("asf-daac", "Update Service", "HKH", "None", "HKH_RTC", None, aid_rtc, "Dedicated Instance",
                 "Radiometric Terrain Corrected (RTC) products generated "
                 "from Sentinel-1 SAR imagery over flood-prone regions in the Hindu Kush Himalayan (HKH) region "
                 "for the 2022 monsoon season, processed by ASF.",
                 "RTC products processed by ASF DAAC HyP3 2022 using GAMMA software. Contains modified Copernicus "
                 "Sentinel data 2022, processed by ESA.", '',
                 False, False, True, None, None, None, None)
print('RTC Image Service updated.')